In [1]:
import numpy as np
import matplotlib as mp
import numpy.linalg as la
from decimal import Decimal
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
import pylab
import math
import os

import networkx as nx
import plotly.graph_objs as go
import scipy
import scipy.linalg as spla
import scipy.sparse
import scipy.optimize
from scipy.interpolate import interp1d
import meshio
#from scipy import interpolate

pgf_with_latex = {                      # setup matplotlib to use latex for output
    "pgf.texsystem": "pdflatex",        # change this if using xetex or lautex
    "text.usetex": True, 
    "pgf.preamble": "\\usepackage{siunitx} \\usepackage{newtxtext,newtxmath}"                              # using this preamble
    }
mp.use("pgf")
mp.rcParams.update(pgf_with_latex)

import meshio as meshio

Definition of the path to the data
Data output used in this project generated using OpenGeoSys

In [2]:
path = 'data/'
err = 1e-04

#35.792 Nonzero
#7.043.716 Werte
#1327 Knoten
#2654 Freiheitsgrade

In [3]:
def getMatrix( pathMatrix ):
    matFile = open( pathMatrix, 'r')
    mat = np.zeros((2654,2654))
    next(matFile)

    #save values of the file in a matrix
    for line in matFile:
        if( line.strip() ):
            splitline = line.split(" ")
            mat[int(splitline[0])][int(splitline[1])] = float(splitline[2])
    matFile.close()
    return mat

def getVector( pathVector ):
    vecFile = open( pathVector, 'r')
    vec = np.zeros((2654))
    next(vecFile)

    #save values of the file in a matrix
    index = 0
    for line in vecFile:
        if( line.strip() ):
            vec[index] = line
            index += 1
    vecFile.close()
    return vec

In [4]:
def getBoundary( points ):
    min_x = 1e+10
    max_x = -1e+10
    min_y = 1e+10
    for i_points in points:
        if( i_points[0] < min_x ):
            min_x = i_points[0]
        elif( i_points[0] > min_x ):
            max_x = i_points[1]
        if( i_points[1] < min_y ):
            min_y = i_points[1]
    return min_x, max_x, min_y

In [5]:
def getEigenmode( mat, numEval ):
    w,v=spla.eig(mat)
    idx = w.argsort()[::-1]   
    w = w[idx]
    v = v[:,idx]

    return w[numEval], v.T[numEval]

def getSortedEigenvalues( mat ):
    w,v=spla.eig(mat)
    idx = w.argsort()[::1]   
    w = w[idx]
    v = v[:,idx]

    return w, v.T

def saveSmallestEigenvalues( path, file ):
    outFile = open( file, "a")
    list = os.listdir(path + 'timestepsReduced')
    list.sort()
    
    for i_path in list:
        mat = getMatrix( path + "timestepsReduced/" + i_path)
        mat = apply_Dirichlet_bc_boundary(mat, points, 0, leftX, rightX, bottomY)
        w,v = getEigenmode( mat, len(mat[0])-1 )
        outFile.write( str(w.real) + "\n")
    outFile.close()

#saveSmallestEigenvalues( path, path + "eigenvalues/smallest.txt" )

In [6]:
def apply_Dirichlet_bc_OGS(K_glob, node_id,value):
    c = K_glob[node_id,node_id]
    K_glob[node_id,:] = 0.
    K_glob[:,node_id] = 0.
    
    if (c != 0):
        K_glob[node_id,node_id] = c
    else: 
        K_glob[node_id,node_id] = 1.
        c = 1.
    return K_glob

def apply_Dirichlet_bc_boundary(K_glob, points, value, leftX, rightX, bottomY):
    solution = np.array([0.]*2*len(points))
    for i_points in range (0, len(points)):
        if( abs(points[i_points][0]- leftX) < err ):
            K_glob = apply_Dirichlet_bc_OGS( K_glob, 2*i_points, value )
        elif( abs(points[i_points][0]- rightX) < err ):
            K_glob = apply_Dirichlet_bc_OGS( K_glob, 2*i_points, value )
        if( abs(points[i_points][1]- bottomY) < err ):
            K_glob = apply_Dirichlet_bc_OGS( K_glob, 2*i_points+1, value )
    return K_glob

In [10]:
# Load the VTU file
mesh = meshio.read(path+"slope_ts_73_t_5.375165.vtu")

# Load the VTU file
mesh = meshio.read(path+"slope_ts_73_t_5.375165.vtu")

# Access points (nodes) from the mesh
points = mesh.points

# Read the stiffness matrix 
mat1 = getMatrix( path + 'timestepsReduced/stiffness_ogs_global_matrix_cnt_006_t_0.1_pcs_0_Jac.mat')
mat2 = getMatrix( path + 'timestepsReduced/stiffness_ogs_global_matrix_cnt_134_t_2_pcs_0_Jac.mat')
mat3 = getMatrix( path + 'timestepsReduced/stiffness_ogs_global_matrix_cnt_624_t_5.37527_pcs_0_Jac.mat')

# get the boundarys of the domain
leftX, rightX, bottomY = getBoundary( points )

mat1 = apply_Dirichlet_bc_boundary(mat1, points, 0, leftX, rightX, bottomY)
mat2 = apply_Dirichlet_bc_boundary(mat2, points, 0, leftX, rightX, bottomY)
mat3 = apply_Dirichlet_bc_boundary(mat3, points, 0, leftX, rightX, bottomY)

Cumulative distribution function of eigenvalues for a list of matrices

In [11]:
def CDF( matList, labelList, colorList ):
    fig, ax = plt.subplots(figsize=(6,3.5))
    for i in range(0,len(matList)):
        w,v = getSortedEigenvalues(matList[i])
        # calculate the proportional values of samples
        p = 1. * np.arange(len(w)) / (len(w) - 1)
        ax.plot(w, p, color=colorList[i], 
                label=labelList[i])# Decrease opacity
    
    # Remove the frame (spines) and leave only left and bottom axes
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_position('zero')
    ax.spines['bottom'].set_position('zero')

    ax.set_xticks(p)
    #ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x)}'))
    ax.set_xlabel("$\\SI[per-mode=symbol]{}{\lambda_0\, \per \, N\metre^{-2}}$")
    ax.set_ylabel("Fraction smaller")
    ax.set_xscale('symlog')
    ax.set_yscale('symlog', linthresh=1e-04)
    ax.set_xlim(xmin = -0.1)
    ax.set_ylim(ymin = 0)
    ax.axhline(y=1e-4, color='black', linestyle= '--')
    legend=fig.legend(ncols=len(matList), loc='upper center')
    legend.get_frame().set_facecolor('none')  # Fully transparent background
    legend.get_frame().set_edgecolor('none')  # No border
    #tikzplotlib.save("CDF.tex")
    fig.tight_layout()
    fig.savefig(path + 'CDF.pdf')

In [12]:
CDF([mat1, mat2, mat3], ["t=0.1 s", "t=2.0 s", "t=5.37527 s"], ['blue', 'green', 'red'])

/home/chiara/my_project_dir/my_project_env/lib/python3.10/site-packages/matplotlib/cbook.py:1762: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
/home/chiara/my_project_dir/my_project_env/lib/python3.10/site-packages/matplotlib/cbook.py:1398: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


In [17]:
def interpolation( fx1, fx2, x ):
    x = x%1
    return fx1+(fx2-fx1)*(x)

Plot the norm of the eigenvector related to a node on the node in deformed state

In [26]:
def plotEigenmode(mat, vtu, numEval, scale, boolEdges, boolEigenMode, saveToFile, jLabel):
    G = nx.Graph()
    from_list = []
    to_list = []
    values = []
    edge_color = []
    mesh = meshio.read(vtu)
    points = mesh.points
    evalue, evec = getEigenmode(mat, numEval)

    if evec[2*72+1].real < 0:
        evec = -1 * evec
    
    value = len(mat[0])
    for i in range(0, int(value/2)):
        G.add_node(i, pos=(points[i][0] + (scale) * evec[2*i].real,
                           points[i][1] + (scale) * evec[2*i+1].real))
        values.append(math.sqrt( pow(evec[2*i].real, 2 ) + pow( evec[2*i+1].real , 2 ) ) )

        for j in range(0, int(i)):
            if (abs(mat[i*2][j*2]) > 1e-04 or
                abs(mat[i*2+1][j*2+1]) > 1e-04 or
                abs(mat[i*2+1][j*2]) > 1e-04 or
                abs(mat[i*2][j*2+1]) > 1e-04):
                from_list.append(i)
                to_list.append(j)

    plt.figure(figsize=(7, 6))
    plt.title(f"$\\lambda^{{ {jLabel} }} = \\text{{ {round(evalue.real, 4)} }} \\,\\text{{Nm}}^{{-2}}$", loc='left')

    if boolEdges == 1:
        for k in range(len(from_list)):
            G.add_edges_from([(from_list[k], to_list[k])])
            edge_color.append(mat[from_list[k]][to_list[k]])

        pos = nx.get_node_attributes(G, 'pos')
        nx.draw_networkx_edges(G, pos, width=0.5, alpha=0.5, edge_color='#888')

    if boolEigenMode == 1:
        pos = nx.get_node_attributes(G, 'pos')
        scatter = plt.scatter([], [], c=[], s=5, cmap='RdYlBu_r', alpha=1, vmin=min(values), vmax=max(values))
        
        for i, node in enumerate(G.nodes()):
            x, y = G.nodes[node]['pos']
            plt.scatter(x, y, c=values[i], s=5, cmap='RdYlBu_r', vmin=min(values), vmax=max(values))

        cbar = plt.colorbar(scatter, ax=plt.gca(), orientation='horizontal', pad=0.05, shrink=0.85)
        cbar.set_label(f"$\\lVert \\textbf{{v}}^{{ {jLabel} }}_i\\rVert$")
        cbar.ax.set_box_aspect(0.02) #make colorbar thinner

    plt.axis('off')

    # Set the aspect ratio of the axes equal
    plt.gca().set_aspect('equal', adjustable='box')

    if saveToFile != '':
        plt.savefig(saveToFile, bbox_inches='tight')
    
    #plt.show()

In [22]:
list = os.listdir(path + 'timestepsReduced')
list.sort()
i = 0
timestep = 0

for i_path in list:
    mat = getMatrix( path + "timestepsReduced/" + i_path)
    mat = apply_Dirichlet_bc_boundary(mat, points, 0, leftX, rightX, bottomY)
    plotEigenmode(mat, path + "slope_ts_74_t_5.375265.vtu", int(len(mat)-1), 10, 1, 1, path + 'timestep' + timestep + '.pdf', 0)
    plotEigenmode(mat, path + "slope_ts_74_t_5.375265.vtu", int(len(mat)-2), 10, 1, 1, path + 'timestep' + timestep + '_second.pdf', 1)
    plotEigenmode(mat, path + "slope_ts_74_t_5.375265.vtu", 0, 10, 1, 1, path + 'timestep' + timestep + '_higher.pdf', 1326)
    
    timestep = timestep + 1;

Plot the displacement/eigenvector in x and y direction
mode: 0-displacement, 1-eigenvector, 2-difference
direction: 0-x, 1-y

In [29]:
def plotDisplacementEigenvalue( mat, vtu, saveToFile, meshData, mode, direction, label ):
    G = nx.Graph()
    from_list = []
    to_list = []
    values = []
    edge_color = []
    mesh = meshio.read(vtu)
    points = mesh.points
    evalue, evec = getEigenmode( mat, len(mat[0])-1 )

    w,v=spla.eig(mat)
    idx = w.argsort()[::-1]   
    w = w[idx]
    v = v[:,idx]

    data = meshio.read(path + meshData)
    displacement = data.point_data["displacement"]

    norm = 0
    for i in range(0,2):
        for j in range(0,len(displacement)):
            norm = norm + pow( displacement[j][i], 2 )
    norm = math.sqrt(norm)
    
    value = len(mat[0])
    for i in range(0, int(value/2)):
        G.add_node( i, data={'label': str( i )}, pos=(points[i][0], points[i][1]) )
        if( mode == 0 ):
            if( direction == 0 ):
                values.append( evec[2*i].real )
            else:
                values.append( evec[2*i+1].real )
        elif( mode == 1 ):
            if( direction == 0 ):
                values.append( displacement[i][0]/norm )  #- evec[2*i+1].real )
            else:
                values.append( displacement[i][1]/norm ) 
        else:
            if( direction == 0 ):
                values.append( displacement[i][0]/norm - evec[2*i].real )
            else:
                values.append( displacement[i][1]/norm - evec[2*i+1].real )
        
        for j in range(0, int(i)):
            if (abs(mat[i*2][j*2]) > 1e-04 or
                abs(mat[i*2+1][j*2+1]) > 1e-04 or
                abs(mat[i*2+1][j*2]) > 1e-04 or
                abs(mat[i*2][j*2+1]) > 1e-04):
                from_list.append(i)
                to_list.append(j)

    plt.figure(figsize=(7, 6))

    for k in range(len(from_list)):
        G.add_edges_from([(from_list[k], to_list[k])])
        edge_color.append(mat[from_list[k]][to_list[k]])

    pos = nx.get_node_attributes(G, 'pos')
    nx.draw_networkx_edges(G, pos, width=0.5, alpha=0.5, edge_color='#888')

    pos = nx.get_node_attributes(G, 'pos')
    scatter = plt.scatter([], [], c=[], s=5, cmap='RdYlBu_r', alpha=1, vmin=min(values), vmax=max(values))
        
    for i, node in enumerate(G.nodes()):
        x, y = G.nodes[node]['pos']
        plt.scatter(x, y, c=values[i], s=5, cmap='RdYlBu_r', vmin=min(values), vmax=max(values))

    cbar = plt.colorbar(scatter, ax=plt.gca(), orientation='horizontal', pad=0.05, shrink=0.85)
    cbar.set_label(label)
    cbar.ax.set_box_aspect(0.02)  # make color bar thinner

    plt.axis('off')

    # Set the aspect ratio of the axes equal
    plt.gca().set_aspect('equal', adjustable='box')

    if saveToFile != '':
        plt.savefig(saveToFile, bbox_inches='tight')
    
    #plt.show()

In [30]:
#label = '$\mathbf{v}_{j_x}^{0}$'
#label = '$u_{j_x} \lVert\mathbf{u}\\rVert^{-1} - v_{j_x}^{0}$'
label = '$u_{j_x} \lVert\mathbf{u}\\rVert^{-1}$'

plotDisplacementEigenvalue( mat3, path + "slope_ts_74_t_5.375265.vtu", path + 'SumEigenmodeUX.pdf', "slope_ts_74_t_5.375265.vtu", 0, 0, label)